In [1]:
%matplotlib inline
import matplotlib.pyplot as plt 
import matplotlib.dates as dt
import pandas as pd 
import numpy as np
from datetime import datetime
from sklearn.svm import LinearSVR

In [2]:
#chargement des données
data_train = pd.read_csv("train.csv") 

In [3]:
data_test = pd.read_csv("test.csv")

In [10]:
#la fonction de calcule de la direction
def bearing_array(lat1, lng1, lat2, lng2):
    lng_delta_rad = np.radians(lng2 - lng1)
    lat1, lng1, lat2, lng2 = map(np.radians, (lat1, lng1, lat2, lng2))
    y = np.sin(lng_delta_rad) * np.cos(lat2)
    x = np.cos(lat1) * np.sin(lat2) - np.sin(lat1) * np.cos(lat2) * np.cos(lng_delta_rad)
    return np.degrees(np.arctan2(y, x))
data_train['direction'] = bearing_array(data_train['pickup_latitude'].values,data_train['pickup_longitude'].values,data_train['dropoff_latitude'].values,data_train['dropoff_longitude'].values)
data_test['direction'] = bearing_array(data_test['pickup_latitude'].values,data_test['pickup_longitude'].values,data_test['dropoff_latitude'].values,data_test['dropoff_longitude'].values)

In [6]:
#le calcule de la distance en utilisant la bibliothéque pyproj
from pyproj import Geod

wgs84_geod = Geod(ellps='WGS84') #Distance will be measured on this ellipsoid - more accurate than a spherical method

#Get distance between pairs of lat-lon points
def Distance(lat1,lon1,lat2,lon2):
    az12,az21,dist = wgs84_geod.inv(lon1,lat1,lon2,lat2) #Yes, this order is correct
    return dist
#Add/update a column to the data frame with the distances (in metres)
data_train['approx_distance'] = Distance(data_train['pickup_latitude'].tolist(),data_train['pickup_longitude'].tolist(),data_train['dropoff_latitude'].tolist(),data_train['dropoff_longitude'].tolist())
data_test['approx_distance'] = Distance(data_test['pickup_latitude'].tolist(),data_test['pickup_longitude'].tolist(),data_test['dropoff_latitude'].tolist(),data_test['dropoff_longitude'].tolist())

In [15]:
#la création de modéle SVR avec les paramétres
regr = LinearSVR(random_state=0, tol=1e-5, epsilon = 1 ,max_iter = 10000)

In [17]:
regr.fit(data_train['approx_distance'].values.reshape(-1, 1), data_train['trip_duration'])

C:\Users\Hatim\Anaconda3\lib\site-packages\sklearn\svm\base.py:929: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  "the number of iterations.", ConvergenceWarning)


LinearSVR(C=1.0, dual=True, epsilon=1, fit_intercept=True,
          intercept_scaling=1.0, loss='epsilon_insensitive', max_iter=10000,
          random_state=0, tol=1e-05, verbose=0)

In [19]:
#verification du score par rapport aux données d'entrainement (SKLEARN SCORE)
regr.score(data_train['approx_distance'].values.reshape(-1, 1), data_train['trip_duration'])

0.002557075165969036

In [20]:
#prédiction de la durée du trajet des données de test et l'écriture des résultats dans le fichier csv
Z=regr.predict(data_test['approx_distance'].values.reshape(-1, 1))
S = pd.DataFrame()
S["id"]= data_test["id"]
S["trip_duration"] = Z
S.to_csv("submittionsvm.csv",index=False)

array([179.00637752])